# Tutorial: Creating and pushing calculated signals

How to push calculated signals and scalars using Seeq Data Lab and manipulate worksheet items.

- **Author:** Siang Lim
- **Date:** June 6th 2022

## Background

**Steam savings** is one component in a typical refinery energy dashboard.

In this notebook, we will demonstrate how to use Seeq Data Lab to calculate steam savings and push the results back to the Workbench, using the following tags in the Splitter as a demo:

- **53FFR412** - Stripping Steam/Bottoms Ratio
- **53FC128** - Bottoms to FCC flow

### Calculations
Steam savings is defined relative to a baseline steam flow to bottoms. For this tag, the baseline ratio is $R_b = 7.1$.

$$
\begin{align}
F, \text{Steam Saved [lb/h]}       &= (R_b - R_s) * F_B \\
C, \text{Steam Cost [\$/(klb/h)]}  &= \$13.51 \quad \text{ (from Planning dept or market prices)}\\
S, \text{Savings [\$/day]}         &= F \text{ [lb/h]} \cdot C \text{ [\$/(klb/h)]} \cdot 24 \text{ [h/day]} \cdot 1/1000 \text{ [(klb/lb)]}
\end{align}
$$

Where,

$$
\begin{align}
R_s &= \text{Current steam/bottoms ratio, 53FFR412, [unitless]} \\
F_B &= \text{Bottoms flow, 53FC128, [lb/h]}
\end{align}
$$

### Set up variables

In [ ]:
R_b = 7.1
cost = 13.51

### Seeq formula
Check out the Seeq training courses if you need a review on Seeq formulas. **Note that:** Seeq variables must start with a letter: `$f128` is valid, `$53ffr412` is not. 

**First, a signal:** Steam saved is a `signal`, and will be defined as follows.

In [ ]:
steam_saved = f"(({R_b}-$f412)*$f128/24).setUnits('lb/h')"
steam_saved

**Aside:** We've used the Python f-string syntax (as indicated by the `f` before the string) to embed expressions inside string. In this case, our expression was `R_b`. For more information on f-strings, see https://realpython.com/python-f-strings/#:~:text=%E2%80%9CF%2Dstrings%20provide%20a%20way,which%20contains%20expressions%20inside%20braces.


**Second, scalar:** Steam cost is a `scalar` (it has a single value), and will be defined as:

In [ ]:
steam_cost = f"({cost}).setunits('$/klb/h')"
steam_cost

**Third, another signal:** Finally, the savings per day is calculated as:

In [ ]:
savings = f"($ss*$co*24*(1/1000)).setunits('$/day').remove(islessthan(0))"
savings

## Variable assignment
We'll need to tell Seeq what those variables are: 

- `$f412`
- `f128`
- `$ss`
- `$co`

Since `$ss` and `$co` in the savings calculation depends on previously calculated values, we will need to push the calculations to Seeq in 2 separate steps.

We will see how to do this below.

## Step 1 - Import libraries

In [ ]:
from seeq import spy
import pandas as pd
pd.set_option('display.max_colwidth', None)

## Step 2 - Define tags and data source

Replace `YOUR PI SERVER` with your PI server.

In [ ]:
my_items = pd.DataFrame({
    'Name': ['53FFR412', '53FC128'],
    'Datasource Name': 'YOUR PI SERVER'
})

my_items

## Step 3 - Search for tags using `spy.search`

In [ ]:
metadata_df = spy.search(my_items)
metadata_df

## Step 4 - First, push PI tags back to Seeq Workbench

You can tell Seeq which workbook and worksheet you want to push the signal back to. Leave it blank and it will push it to a default SDL workbook called `Data Lab >> Data Lab Analysis` and worksheet called `From Data Lab`.

More information can be found in the SPy.push documentation below using `help(spy.push)`.

In [ ]:
help(spy.push)

In [ ]:
workbook_ID = '7836C665-2B4C-4B36-8262-BE5230E102A5' # Change this to where you want it
worksheet_name = 'Dev 1'

In [ ]:
push_results = spy.push(
    workbook=workbook_ID,
    worksheet=worksheet_name,
    metadata=metadata_df)

push_results

> Navigate to the link above to see the results. You may need to replace the IP address with your Seeq server if it's not pointing to the right place (still trying to figure out how to fix this)

#### WRONG: 
- http://`1.2.3.4`/892DF617-6104-4BFF-BCFC-6256FE4DFA7A/workbook/7836C665-2B4C-4B36-8262-BE5230E102A5/worksheet/9F20FD2B-F56E-4D04-8FCB-FF9A661EA39D

#### RIGHT: 
- http://`MYSEEQSERVER`/892DF617-6104-4BFF-BCFC-6256FE4DFA7A/workbook/7836C665-2B4C-4B36-8262-BE5230E102A5/worksheet/9F20FD2B-F56E-4D04-8FCB-FF9A661EA39D

## Step 4 - Define calculations
If you have multi-step calculations where a step depends on a previous calculation, you will need to break them down into several calculations.

Here, we will do the steam saved and cost of steam first. In `Formula Parameters`, we are telling Seeq to grab the IDs of `53FFR412` and `53FC128` to assign tags to the variables we've defined.

> Notice that we passed an entire DataFrame row to it, Seeq will automatically parse the row to find the IDs. We could've also passed the `metadata_df` row instead of `push_results`. However, we want to display all the signals (tags + calculated values), which we will see later in this notebook. Using `push_results` makes this step easier to understand (code-wise).

In [1]:
calc_signals = [{'Name': 'Steam Saved',
                'Type': 'Signal',
                'Formula': steam_saved,
                'Formula Parameters': {'$f412': push_results[push_results['Name'] == '53FFR412'],
                                       '$f128': push_results[push_results['Name'] == '53FC128'],
                                      }
               },
               {'Name': 'Cost of Steam',
                'Type': 'Scalar',
                'Formula': steam_cost
               }]
df_calcs = pd.DataFrame(calc_signals)
df_calcs


NameError: name 'steam_saved' is not defined

Error found at line 3 in cell 1.


Button(description='Click to show stack trace', layout=Layout(height='auto', width='auto'), style=ButtonStyle(…

## Step 6 - Push first 2 calculations to Seeq 
Now we push the first 2 calcs, then store the results as `push_results_2`.

In [ ]:
push_results_2 = spy.push(
    workbook=workbook_ID,
    worksheet=worksheet_name,
    metadata=df_calcs)

push_results_2

## Step 6 - Push third calculation to Seeq 
Notice in `push_results_2`, we now have IDs for the steam saved and steam cost variables. Now we can push the savings signal and then store the results as `push_results_3`.

In [ ]:
calc_signals = [{'Name': 'Savings per Day',
                'Type': 'Signal',
                'Formula': savings,
                'Formula Parameters': {'$ss': push_results_2[push_results_2['Name'] == 'Steam Saved'],
                                       '$co': push_results_2[push_results_2['Name'] == 'Cost of Steam'],
                                      }
               }]
df_calcs = pd.DataFrame(calc_signals)
df_calcs

In [ ]:
push_results_3 = spy.push(
    workbook=workbook_ID,
    worksheet=worksheet_name,
    metadata=df_calcs)

push_results_3

## Step 7 - Getting all signals displayed (Method 1)
You may have noticed that every time we pushed a signal back to the worksheet, the earlier pushed signals are no longer displayed in the Workbench. However, they are still available in the 'Recently Accessed' menu.

To make Seeq display ALL signals, we just need to combine all 3 `push_results` DataFrame and push all signals together. There are, of course, other ways to do this, that may be computationally more efficient, but this method seems to be the most straightforward (code-wise, and to understand what's going on for novice users).

Use `pd.concat()` to merge the 3 dataframes

In [ ]:
df_combined = pd.concat([push_results, push_results_2, push_results_3]).reset_index(drop=True)
df_combined

In [ ]:
push_results_final = spy.push(
    workbook=workbook_ID,
    worksheet=worksheet_name,
    metadata=df_combined)

push_results_final

> Take a look at the Workbench, you will see that all 5 signals are now displayed.

## Step 8 - Getting all signals displayed (Method 2)

Alternatively, you could also pull the worksheet using `spy.workbooks` and then modify the `display_items`.

In [ ]:
workbooks_df = spy.workbooks.search({
    'ID': workbook_ID
})

workbooks_df

In [ ]:
workbooks = spy.workbooks.pull(workbooks_df)
workbooks

Check out the worksheets

In [ ]:
workbooks[0].worksheets

We see that the worksheet we want is the 3rd one. Let's look at the display items:

In [ ]:
worksheet_items = workbooks[0].worksheets[2].display_items
worksheet_items

It shows all 5 signals as expected. As long as we know the name and ID of a signal (tag), we can add it to the display. You could also remove a signal by removing a row here.

Let's remove the first 2 signals then push it back.

In [ ]:
new_worksheet_items = worksheet_items.loc[2:,:]
new_worksheet_items

## Reassign the dataframe

In [ ]:
workbooks[0].worksheets[2].display_items = new_worksheet_items

## Then push it back to the workbench

In [ ]:
spy.workbooks.push(workbooks)

> You should see that the first 2 signals are gone now, if you navigate to the URL listed in the table above (last column)